In [2]:
from pathlib import Path
from dotenv import load_dotenv
import polars as pl
import openai
import os

%load_ext dotenv
%dotenv ../openaihelper/.env

# Client Creation

In [7]:
print(f'OPENAI_ORG_ID:\t{os.environ["OPENAI_ORG_ID"][:4]}')
print(f'OPENAI_PROJ_ID:\t{os.environ["OPENAI_PROJ_ID"][:5]}')
print(f'OPENAI_API_KEY:\t{os.environ["OPENAI_API_KEY"][:3]}')

OPENAI_ORG_ID:	org-
OPENAI_PROJ_ID:	proj_
OPENAI_API_KEY:	sk-


In [3]:
client = openai.OpenAI(
    # organization=os.environ["OPENAI_ORG_ID"],
    # project=os.environ["OPENAI_PROJ_ID"],
    # api_key=os.environ["OPENAI_API_KEY"],
)

# Text Generation

In [4]:
messages = [
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."},
]

response = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0.7)
print(response.choices[0].message.content)

In the realm of code where logic flows,  
There lies a dance, where mystery grows,  
A concept profound, like whispers in air,  
Let me weave you a tale of recursion, rare.  

Imagine a mirror, reflecting your face,  
It shows you again, in an infinite space,  
Each glance reveals what’s already been shown,  
A loop of reflections, all on its own.  

In programming’s heart, recursion takes flight,  
A function calls itself, a curious sight,  
To solve a grand puzzle, it breaks it apart,  
Like nesting dolls, each holds its own heart.  

First, a base case—you must clearly define,  
A stopping condition, where all will align,  
For without this anchor, the calls will cascade,  
Into depths of confusion, where chaos is laid.  

With each little call, the problem grows small,  
Reducing its size, until it’s nothing at all,  
Then back through the layers, the answers return,  
A tapestry woven, as lessons we learn.  

So, here’s to recursion, both elegant and neat,  
A spiral of logic, a r

# Vision Analysis

In [5]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/3/31/San_Lorenzo_Monument_4_crop.jpg",
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The image depicts a large stone head sculpture, likely a famous Olmec colossal head. These heads are characteristic of the Olmec civilization, known for their distinct features and found primarily in Mexico. The sculpture typically represents a ruler or an important figure, showcasing intricate details such as the use of headdresses and facial expressions.', refusal=None, role='assistant', function_call=None, tool_calls=None))


# JSON Output

In [6]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    seed=42,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": "What are the objects mentioned in the following sentence, and their colors?: I have a blue cup and a green plate."},
    ],
)
print(response.choices[0].message.content)

{
  "objects": [
    {
      "object": "cup",
      "color": "blue"
    },
    {
      "object": "plate",
      "color": "green"
    }
  ]
}


## Process Files in a Directory

In [13]:
import logging
import json
from pathlib import Path
from datetime import datetime as dt
import openai

# log output to a file and to console
logger = logging.getLogger("openai")
logging.basicConfig(filename=f'openai-{dt.now()}.log', encoding='utf-8', level=logging.INFO)

# change the system prompt to whatever you want
system_message = """
I study discretionary compensation (DC) in executive compensation. My research focuses on both positive and negative adjustments within DC. I analyze sentences from SEC Form DEF 14A to determine the use of DC.

  Extract and determine:
  - `discomp`: 1 if discretionary compensation is exercised, 0 otherwise.
  - `posdiscomp`: 1 if positive discretion is exercised (when `discomp` = 1), 0 otherwise.
  - `negdiscomp`: -1 if negative discretion is exercised (when `discomp` = 1), 0 otherwise.
  - `why`: Extract sentences justifying the above values.
  
  Please provide the results in a JSON format.

"""

client = openai.OpenAI()

in_path = Path("./data/input")
out_path = Path("./data/output")
for file in in_path.glob("*.txt"):
    try:
        text = file.read_text()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            seed=42,
            temperature=0,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": f'"{text}"'},
            ],
        )
        logger.info(f"processed file: {file.name}")
        out_file = out_path / f"{file.stem}.json"
        out_file.write_text(response.to_json())

    except Exception as e:
        logger.error(f"error processing file: {file.name}")
        logger.error(e)


In [17]:
response_json = json.loads(out_file.read_text())
response_json


{'id': 'chatcmpl-AAkXXNZnVwOUfugcE6Lf5FPIRP6dE',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{\n  "discomp": 1,\n  "posdiscomp": 1,\n  "negdiscomp": 0,\n  "why": [\n    "The Compensation Committee approved shifting a portion of our executives’ fixed total direct compensation to base salary, with a corresponding reduction to their LTIP target opportunities.",\n    "Mr. Isom’s target bonus opportunity was increased from 175% to 200%, to reflect his promotion to Chief Executive Officer, consistent with the level previously in place for Mr. Parker.",\n    "Consistent with our emphasis on pay for performance and our commitment to long-term value creation for our stockholders, our named executive officers’ total target direct compensation is weighted heavily toward long-term equity awards.",\n    "Due to the CARES Act, PSP2 and PSP3 compensation limits applicable to our named executive officers, the target values of each of the 2022 L

In [19]:
json.loads(response_json['choices'][0]['message']['content'])

{'discomp': 1,
 'posdiscomp': 1,
 'negdiscomp': 0,
 'why': ['The Compensation Committee approved shifting a portion of our executives’ fixed total direct compensation to base salary, with a corresponding reduction to their LTIP target opportunities.',
  'Mr. Isom’s target bonus opportunity was increased from 175% to 200%, to reflect his promotion to Chief Executive Officer, consistent with the level previously in place for Mr. Parker.',
  'Consistent with our emphasis on pay for performance and our commitment to long-term value creation for our stockholders, our named executive officers’ total target direct compensation is weighted heavily toward long-term equity awards.',
  'Due to the CARES Act, PSP2 and PSP3 compensation limits applicable to our named executive officers, the target values of each of the 2022 LTIP awards continued to be significantly reduced, with Mr. Isom’s LTIP target value reduced over 20%, as compared with 2020 levels.',
  'The Compensation Committee is evaluatin